# SWMAL Exercise


## Convolutional Neural Networks (CNNs)
<details>
For the journal: 

* write an introduction to CNNs (what are CNNs, what is a convolution layer, etc..), 
* document your experiments towards the end-goal of reaching 'a high accuracy' (what did you try, what work/did not work), 
* document how you use '_generalization_' in your setup (us of simple hold-out/train-test split or k-fold, or etc..),
* produce some sort of '_learning-curve_' that illustrates the drop in cost- or increase in score-function with respect to, say training iteration (for inspiration see fig 4.20, 10-12 or 10.17 in [HOML]),
* document the final CNN setup (layers etc., perhaps as a graph/drawing), 
* discus on your iterations towards the end-goal and other findings you had,
* and, as always, write a conclusion.

If you use a code template from slides, HOML or the internet, then remember to add a reference to the original work in you journal.
</details>
Excercise 9 from [HOML], p.496 2nd./p.535 3rd. (slighty modified):

__"9. Build an CNN via the Keras API and try to achieve the highest possible accuracy on MNIST."__


This is from DUWE... rewrite still needed and explaination 

In [3]:
import keras
import numpy as np
from sklearn.model_selection import StratifiedKFold

(x_train, y_train ), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = x_train.reshape((-1,28,28,1)).astype('float32') / 255.0
x_test = x_test.reshape((-1,28,28,1)).astype('float32') / 255.0

y_train = keras.utils.to_categorical(y_train, num_classes=10)
y_test = keras.utils.to_categorical(y_test, num_classes=10)

def create():
    model = keras.models.Sequential()
    model.add(keras.layers.Conv2D(32,(3,3), activation='relu', input_shape=(28,28,1)))
    model.add(keras.layers.MaxPooling2D((2, 2)))
    model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(keras.layers.MaxPooling2D((2, 2)))
    model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dense(10, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
    return model

folds = 5
kfold = StratifiedKFold(n_splits=folds, shuffle=True, random_state=50)
test_accuracy = []

for train, val in kfold.split(x_train,y_train.argmax(1)):

    model = create()

    x_train_kfold, x_val_kfold = x_train[train], x_train[val]
    y_train_kfold, y_val_kfold = y_train[train], y_train[val]

    model.fit(x_train_kfold, y_train_kfold, epochs=20, batch_size=64, verbose=0)

    test_loss, test_accuracy = model.evaluate(x_test,y_test)

avg_accuracy = np.mean(test_accuracy)
print(f'Test average accuracy: {avg_accuracy}')

Epoch 1/20


ValueError: in user code:

    File "c:\Users\Bjark\source\repos\MAL\MAL_V11\Lib\site-packages\keras\src\engine\training.py", line 1377, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\Bjark\source\repos\MAL\MAL_V11\Lib\site-packages\keras\src\engine\training.py", line 1360, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\Bjark\source\repos\MAL\MAL_V11\Lib\site-packages\keras\src\engine\training.py", line 1349, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\Bjark\source\repos\MAL\MAL_V11\Lib\site-packages\keras\src\engine\training.py", line 1127, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\Users\Bjark\source\repos\MAL\MAL_V11\Lib\site-packages\keras\src\engine\training.py", line 1185, in compute_loss
        return self.compiled_loss(
    File "c:\Users\Bjark\source\repos\MAL\MAL_V11\Lib\site-packages\keras\src\en